In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

tqdm.pandas()

In [2]:
page = requests.get("https://beallslist.net/")
soup = BeautifulSoup(page.content, "html.parser")

In [3]:
for ul in soup.find_all("ul", class_="main-menu"):
    ul.decompose()
    # print(ul.prettify())

In [4]:
for ul in soup.find_all("ul", class_="sub-menu"):
    ul.decompose()
    # print(ul.prettify())

In [5]:
links = soup.select("ul > li > a")

names = []
urls = []

for link in links:
    names.append(link.contents[0])
    urls.append(link.get("href"))

df = pd.DataFrame({"source": names, "url": urls})

In [6]:
# last 2 rows redundant
df.drop(index=df.index[-2:], axis=0, inplace=True)
df

,source,url
0,1088 Email Press,http://1088.email/
1,2425 Publishers,http://2425publishers.org/
2,The 5th Publisher,http://www.5thpublisher.com/
3,ABC Journals,http://www.abcjournals.net/
4,A M Publishers,http://ampublisher.com/index.html
...,...,...
1375,"United Prime Publications, Universal Access Me...",https://www.untdprimepub.com/
1376,Universal Wiser Publisher,http://www.wiserpub.com/
1377,Vision Science Research Sdn Bhd,http://vision-science.com/
1378,Universe Scientific Publishing,http://www.usp-pl.com/


In [7]:
def url_ok(url):
    try:
        r = requests.head(url, timeout=30)
        return r.status_code
    except:
        return None

# def url_ok2(url):
#     try:
#         return urllib.request.urlopen(url).getcode()
#     except:
#         return None

In [8]:
df["status"] = df.progress_apply(lambda row: url_ok(row["url"]), axis=1)

100%|██████████| 1380/1380 [22:52<00:00,  1.01it/s] 


In [9]:
df["status"].value_counts(dropna=False, normalize=True)

200.0    0.363768
301.0    0.281884
NaN      0.112319
406.0    0.099275
302.0    0.060145
404.0    0.034058
403.0    0.031159
500.0    0.007971
400.0    0.003623
503.0    0.002174
412.0    0.001449
410.0    0.000725
408.0    0.000725
501.0    0.000725
Name: status, dtype: float64

In [10]:
df["date"] = datetime.today().strftime('%B %d, %Y')

In [11]:
df.to_csv("../data/df_weekly_publishers.csv", encoding='utf-8', index=False)